### The main purpose of this notebook is to generate a mapping that suits the whole dataset

In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
train_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set'
test_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/test set'
full_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data'

In [3]:
gc.collect()

5

## Part A. Demo Converting Text File into Numpy Array 

In [4]:
# path_file = path_folder + '/20180624paths'

In [5]:
'''
results = []
with open(path_file) as inputfile:
    for line in inputfile:
        results.append(line.strip().split('|'))
'''

"\nresults = []\nwith open(path_file) as inputfile:\n    for line in inputfile:\n        results.append(line.strip().split('|'))\n"

In [6]:
# results

In [7]:
'''
valid_results = []
for row in results:
    if ',' in row[2]:
        valid_results.append(row)
'''        

"\nvalid_results = []\nfor row in results:\n    if ',' in row[2]:\n        valid_results.append(row)\n"

In [8]:
# valid_results

As shown, among 90551 paths in file 0624, only 20706 paths got more than 1 item presented

In [9]:
'''
arr = np.array(results)
val_arr = np.array(valid_results)
arr.shape, val_arr.shape
'''

'\narr = np.array(results)\nval_arr = np.array(valid_results)\narr.shape, val_arr.shape\n'

## Part B. Process all the Path Files in Folder

In [5]:
def txt2list(txt_path):
    '''
    input:
        1. txt_path: path to the text file
    output:
        2. a list containing individual lists, each of which contains:
            userid, pathid, itemids in path
    '''
    results = []
    with open(txt_path) as inputfile:
        for line in inputfile:
            results.append(line.strip().split('|'))
            
    valid_results = []
    for row in results:
        if ',' in row[2]:
            valid_results.append(row)
            
    return valid_results

In [6]:
from os import listdir
from os.path import isfile, join
def get_paths_from_folder(folder_path):    
    return [join(folder_path, f) for f in listdir(folder_path) if isfile(join(folder_path, f))]

In [7]:
file_paths = get_paths_from_folder(full_folder)
file_paths

['/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/.DS_Store',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-06-29',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-06-30',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-07-01',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-07-02',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-07-03',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-07-04',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-07-05',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data/2018-0

In [8]:
all_paths = []
for txt_path in file_paths:
    if not txt_path.endswith('.DS_Store'):
        all_paths = all_paths + txt2list(txt_path)
len(all_paths)    

2365418

In [9]:
arr = np.array(all_paths)
arr = arr.astype(object)
arr.shape

(2365418, 3)

## Part C. Sort the Paths by Lengths

In [10]:
for j in range(len(arr)):
    arr[j][0] = int(arr[j][0])
    arr[j][1] = int(arr[j][1])
    arr[j][2] = list(arr[j][2].split(','))
    for i in range(len(arr[j][2])):
        arr[j][2][i] = int(arr[j][2][i])
        
arr

array([[185274, 13, list([64814240, 1036940038, 1167902574, 507765172])],
       [590122, 8, list([377314456, 1211467325])],
       [802311, 19, list([1107798762, 1245925754, 1245565340])],
       ...,
       [76346846, 12, list([702374141, 675615375])],
       [76455160, 26,
        list([1061477831, 706672870, 1132726214, 1092471096])],
       [76530139, 0, list([682505284, 642637406])]], dtype=object)

In [11]:
arr[0]

array([185274, 13, list([64814240, 1036940038, 1167902574, 507765172])],
      dtype=object)

In [12]:
# create a list containing the session lengths

sess_len_list = []
for i in range(arr.shape[0]):
    sess_len_list.append(len(arr[i][2]))
sess_len_arr = np.array(sess_len_list)
sess_len_arr = sess_len_arr.reshape([len(sess_len_list),1])

In [13]:
arr = np.concatenate((arr,sess_len_arr),axis = 1)
arr.shape

(2365418, 4)

In [14]:
arr[0],arr[1]

(array([185274, 13, list([64814240, 1036940038, 1167902574, 507765172]), 4],
       dtype=object),
 array([590122, 8, list([377314456, 1211467325]), 2], dtype=object))

#### Remarks:

Here it is necessary to note that the four columns (0-3) of the numpy array is:

0: usedid as int

1: pathid as int

2: path as np array

3: len(path) as int


In [15]:
df = pd.DataFrame(arr)
df.columns = ['userid', 'sess_id', 'sess_path', 'sess_length']

In [16]:
df = df.sort_values(by = 'sess_length')
df = df.reset_index(drop = 'True')

#### Remarks:
Here a small trouble is encountered: the pathid got duplication, but if we apply drop duplicate, disaster can happen

In [17]:
## df.drop_duplicates(subset='sess_id', keep = 'first')

In [18]:
summary = df.groupby('sess_length').count()
summary = pd.DataFrame(summary.drop(columns=['sess_id','sess_path']))
summary = summary.reset_index()
summary.columns = ['sess_length', 'count']

In [19]:
slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
slice_table

,sess_length,count,start_index,end_index
0,2,1592038,1592038,1592038
1,3,392060,392060,392060
2,4,153887,153887,153887
3,5,78750,78750,78750
4,6,44196,44196,44196
5,7,27154,27154,27154
6,8,18229,18229,18229
7,9,13102,13102,13102
8,10,8727,8727,8727
9,11,7256,7256,7256


In [20]:
# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0

slice_table

,sess_length,count,start_index,end_index
0,2,1592038,0,1592038
1,3,392060,1592038,1984098
2,4,153887,1984098,2137985
3,5,78750,2137985,2216735
4,6,44196,2216735,2260931
5,7,27154,2260931,2288085
6,8,18229,2288085,2306314
7,9,13102,2306314,2319416
8,10,8727,2319416,2328143
9,11,7256,2328143,2335399


#### Planning A:
Here it we can wrap the counting of each session length into a dictionary, when the training batch is randomly generated, the prob of session length to be selected should be proportion to the session length count.

Therefore, the index slicing range for each session length should be presented.

#### Planning B:
Use zero padding + bucketing Method to deal with the data: with different length: the candidate lengths will be: (2,3,4,5,6,10,20,50), sessions with longer length will be discarded

#### Another Planning:
The FM user/item embedding can actually be input into another RNN, say if the item/user embedding DNE: create random embedding.

(Then is it possible to train user/item embedding at RNN???

#### Some Dictionaries Needed:

In the set of active items (items that are presenting in the session paths), the item would have a unique id in this active set

so we need to map: itemid (given) ---> unique id in active item set ---> FM pretrained embedding

so we need to map: userid (given) ---> unique id in active user set --->

## Part D. User/Item Index Mapping

In [21]:
df

,userid,sess_id,sess_path,sess_length
0,220047,7,"[965939533, 953183016]",2
1,45272611,30,"[1074442914, 1232376584]",2
2,43491500,14,"[972868200, 1237454697]",2
3,43240102,23,"[1152943774, 1124644670]",2
4,42295322,23,"[845635126, 845664986]",2
5,41662617,14,"[1189836286, 511241811]",2
6,41093297,2,"[66741389, 1057175182]",2
7,39327103,0,"[1285772268, 1233828716]",2
8,34924417,18,"[1291881508, 1254007367]",2
9,34401987,67,"[1066962261, 1078264215]",2


In [1]:
# return the slicings, which will be used to separate different length of sequences
arr = df.values

NameError: name 'df' is not defined

In [23]:
arr.shape

(2365418, 4)

In [24]:
type(df['sess_path'][1][1])

int

In [25]:
df.head()

,userid,sess_id,sess_path,sess_length
0,220047,7,"[965939533, 953183016]",2
1,45272611,30,"[1074442914, 1232376584]",2
2,43491500,14,"[972868200, 1237454697]",2
3,43240102,23,"[1152943774, 1124644670]",2
4,42295322,23,"[845635126, 845664986]",2


In [26]:
import time

In [ ]:
print(time.ctime())

# now we gonna count the active users and active items
userid_list = []
itemid_list = []
for i in range(arr.shape[0]):
    userid_list.append(arr[i][0])
    itemid_list = itemid_list + arr[i][2]
    
userid_set = set(userid_list)
itemid_set = set(itemid_list)

print('Total amount of user presented before and after dropping duplication:')
print(len(userid_list), len(userid_set))
print('Total amount of item presented before and after dropping duplication:')
print(len(itemid_list), len(itemid_set))

print(time.ctime())

Mon Sep  3 18:30:17 2018


In [1]:
390642/41653

9.378484142798838

In [33]:
userid_df = pd.DataFrame([0]+list(userid_set))
itemid_df = pd.DataFrame([0]+list(itemid_set))
userid_df.columns = ['userid']
itemid_df.columns = ['itemid']
userid_df['userid'] = pd.to_numeric(userid_df['userid'])
itemid_df['itemid'] = pd.to_numeric(itemid_df['itemid'])

userid_df = userid_df.sort_values(by = 'userid').reset_index(drop=True)
itemid_df = itemid_df.sort_values(by = 'itemid').reset_index(drop=True)

In [34]:
pro_data_fold = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Processed Data'

In [35]:
userid_df.to_csv(pro_data_fold + '/userid_map.csv')
itemid_df.to_csv(pro_data_fold + '/itemid_map.csv')

In [36]:
# below is the dictionary for mapping item/user ids in shopee db into model based id

userid_dict = dict([(userid_df['userid'][i], i) for i in range(len(userid_df))])
itemid_dict = dict([(itemid_df['itemid'][i], i) for i in range(len(itemid_df))])

In [37]:
userid_dict

{0: 0,
 10002: 1,
 10026: 2,
 10028: 3,
 10050: 4,
 10057: 5,
 10058: 6,
 10095: 7,
 10107: 8,
 10108: 9,
 10116: 10,
 10124: 11,
 10147: 12,
 10155: 13,
 10159: 14,
 10174: 15,
 10214: 16,
 10268: 17,
 10287: 18,
 10290: 19,
 10295: 20,
 10308: 21,
 10310: 22,
 10359: 23,
 10360: 24,
 10370: 25,
 10378: 26,
 10388: 27,
 10394: 28,
 10398: 29,
 10420: 30,
 10461: 31,
 10484: 32,
 10509: 33,
 10520: 34,
 10523: 35,
 10530: 36,
 10536: 37,
 10556: 38,
 10563: 39,
 11117: 40,
 11122: 41,
 11171: 42,
 11271: 43,
 11285: 44,
 11304: 45,
 11351: 46,
 11355: 47,
 11358: 48,
 11405: 49,
 11465: 50,
 11570: 51,
 11671: 52,
 11702: 53,
 11714: 54,
 11733: 55,
 11918: 56,
 11921: 57,
 12520: 58,
 12521: 59,
 12571: 60,
 12584: 61,
 12597: 62,
 12672: 63,
 12695: 64,
 12713: 65,
 12763: 66,
 12801: 67,
 12805: 68,
 12816: 69,
 13140: 70,
 13214: 71,
 13253: 72,
 13277: 73,
 13313: 74,
 13870: 75,
 13960: 76,
 14061: 77,
 14062: 78,
 14125: 79,
 14683: 80,
 14710: 81,
 14750: 82,
 14869: 83,
 14930

In [417]:
for i in range(arr.shape[0]):
    arr[i][0] = userid_dict[arr[i][0]]
    for j in range(len(arr[i][2])):
        arr[i][2][j] = itemid_dict[arr[i][2][j]]

In [418]:
arr.shape

(301510, 4)

In [419]:
# just to show the ids in the array has been well transferred
arr[290000],arr[289999],arr[301509]

(array([7108, 13,
        list([199969, 65097, 175806, 106668, 103224, 149463, 86365]), 7],
       dtype=object), array([20670, 9,
        list([88375, 47497, 123764, 112823, 53320, 152456, 18853]), 7],
       dtype=object), array([3133, 19,
        list([69798, 5283, 7870, 55844, 4470, 83905, 101160, 60811, 60409, 60312, 22780, 24359, 5334, 63942, 77251, 35613, 15634, 50157, 16532, 11540, 16527, 16510, 16490, 20170, 27257, 27290, 11537, 19389, 70893, 75410, 62562, 75006, 28918, 20172, 6788, 66157, 28549, 32005, 50222, 47180, 34801, 34823, 35898, 10781, 676, 7164, 36033, 26440, 42214, 4855, 26814, 32995, 33405, 11840, 11033, 25315, 33377, 24472, 26556, 27711, 25467, 27881, 29241, 3783, 4407, 23324, 30576, 23079, 17375, 26069, 21927, 29736, 28587, 5374, 29312, 17573, 207, 17447, 13112, 10027, 1741, 27873, 6055, 24747, 19647, 12994, 3006, 6724, 25165, 12800, 8272, 22131, 19041, 25065, 9242, 21350, 10530, 16619, 20609, 22388, 15880, 21394, 21385, 10686, 1440, 699, 1575, 20, 6161, 2685, 34

## Part D. Data Padding & Bucketing

In [420]:
Y_list = []
for i in range(arr.shape[0]):
    Y_list.append(arr[i][2][-1])
    arr[i][2] = arr[i][2][:-1]
Y_arr = np.array(Y_list)
Y_arr = Y_arr.reshape([len(Y_list),1])

arr = np.concatenate((arr,Y_arr),axis = 1)
arr.shape

(301510, 5)

In [421]:
arr

array([[35863, 10, list([144506]), 2, 60286],
       [13691, 12, list([25622]), 2, 39979],
       [13731, 11, list([114933]), 2, 184904],
       ...,
       [13594, 81,
        list([54525, 34886, 61307, 130226, 130225, 20136, 148560, 110995, 38294, 85047, 98436, 155296, 155293, 63525, 11480, 130398, 149263, 111733, 111722, 40847, 89078, 14280, 27207, 33290, 33304, 7542, 9145, 113624, 159997, 109851, 71966, 1978, 9376, 4875, 138910, 115066, 171960, 7371, 6891, 23825, 99161, 63378, 39967, 163618, 28446, 58308, 93562, 109705, 61442, 8227, 166182, 163879, 41640, 22422, 96402, 160336, 30828, 88918, 101293, 101292, 99051, 107968, 49749, 11938, 5785, 92997, 8508, 33208, 31236, 33278, 26889, 13066, 51365, 29541, 98993, 116958, 29290, 165525, 17498, 54285, 155268, 51156, 101336, 11484, 96480, 11936, 32596, 78255, 25547, 40589, 196437, 192604]),
        93, 17489],
       [13594, 92,
        list([54525, 34886, 61307, 130226, 130225, 20136, 148560, 110995, 38294, 85047, 98436, 155296, 155293, 6

In [423]:
# we would directly process the numpy array that contains our data
# the numpy array has been processed with proper mapping
# we would replace the lists with a numpy array

buckets_list = [2,3,4,5,6,10,20,50,100]
# we deduct all elements in the buckets_list by 1, to fulfill the length of path without Y item
for i in range(len(buckets_list)):
    buckets_list[i] = buckets_list[i] - 1
    

In [425]:
def pad2buckets(buckets_list, in_arr):
    '''
    Input:
        1. buckets_list: a list of ints, indicating the step lengths of data we want to generate
        2. data_arr: the half-processed data array
    Output: 
        1. pro_arr: the processed array
    '''
    data_arr = in_arr
    # cursor labels the current max length of our training data
    cursor = 0
    for i in range(data_arr.shape[0]):
        # the fourth (index 3) column is the session length of the current user path
        if len(data_arr[i][2]) > buckets_list[-1]:
            break
        # move the cursor to the correct place, 
        # by right the max_length should be bigger or equal to the length of the current row of data
        while len(data_arr[i][2]) > buckets_list[cursor]:
            cursor = cursor + 1
        
        if len(data_arr[i][2]) == buckets_list[cursor]:
            data_arr[i][2] = np.array(data_arr[i][2])
        if len(data_arr[i][2]) < buckets_list[cursor]:
            # create a temp numpy array 
            #temp_path = np.array([0 for i in range(buckets_list[cursor])])
            for j in range(buckets_list[cursor]-len(data_arr[i][2])):
                # temp_path[j] = data_arr[i][2][j]
                data_arr[i][2].append(0)
            # data_arr[i][2] = temp_path
            data_arr[i][2] = np.array(data_arr[i][2])
            data_arr[i][3] = buckets_list[cursor] + 1
    print(i)       
    # now i is at the first row we want to abandon
    return data_arr[:i]

In [426]:
pro_arr = pad2buckets(buckets_list, arr)

301509


In [427]:
pro_df = pd.DataFrame(pro_arr)
pro_df.columns = ['userid', 'pathid', 'path', 'sess_length', 'Y']

In [428]:
pro_arr


array([[35863, 10, array([144506]), 2, 60286],
       [13691, 12, array([25622]), 2, 39979],
       [13731, 11, array([114933]), 2, 184904],
       ...,
       [13594, 91,
        array([ 54525,  34886,  61307, 130226, 130225,  20136, 148560, 110995,
        38294,  85047,  98436, 155296, 155293,  63525,  11480, 130398,
       149263, 111733, 111722,  40847,  89078,  14280,  27207,  33290,
        33304,   7542,   9145, 113624, 159997, 109851,  71966,   1978,
         9376,   4875, 138910, 115066, 171960,   7371,   6891,  23825,
        99161,  63378,  39967, 163618,  28446,  58308,  93562, 109705,
        61442,   8227, 166182, 163879,  41640,  22422,  96402, 160336,
        30828,  88918, 101293, 101292,  99051, 107968,  49749,  11938,
         5785,  92997,   8508,  33208,  31236,  33278,  26889,  13066,
        51365,  29541,  98993, 116958,  29290, 165525,  17498,  54285,
       155268,  51156, 101336,  11484,  96480,  11936,  32596,  78255,
        25547,  40589, 196437, 192604, 

In [430]:
# derive the summary/slice table for the training data

summary = pro_df.groupby('sess_length').count()
summary = pd.DataFrame(summary.drop(columns=['pathid','path','Y']))
summary = summary.reset_index()
summary.columns = ['sess_length', 'count']

slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0

slice_table

,sess_length,count,start_index,end_index
0,2,202142,0,202142
1,3,50820,202142,252962
2,4,20257,252962,273219
3,5,10362,273219,283581
4,6,5402,283581,288983
5,10,8525,288983,297508
6,20,3174,297508,300682
7,50,727,300682,301409
8,100,100,301409,301509


### Save Essential Utilities

In [1]:
pro_data_fold = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Processed Data'

In [435]:
np.save(pro_data_fold+'/X_train.npy', pro_arr)

## Part E. Demo Data Generator

#### List of Utilities:
1. slice_table as df (can be generated at run time)
2. arr as numpy array: the source of training data

#### Desired Output:
1. Training item numpy array with selected session length
2. Training user numpy array
3. Training label Y

all the above are supposed to come with the same batch_size 

#### Remarks:
1. The probability of each session length to be selected as training data should be proportional to the amount of session with that length presented

#### About the GRU Model Placeholders:
1. y: label place holder: shape is num_item, generated by one hot encoding 
2. itemid (mapped): shape of (batch_size, )
3. userid (mapped): shape of (batch_size, )

#### Remarks:
1. Go check the loss function

In [4]:
pro_arr = np.load(pro_data_fold+'/X_train.npy')

In [5]:
pro_arr

array([[35863, 10, array([144506]), 2, 60286],
       [13691, 12, array([25622]), 2, 39979],
       [13731, 11, array([114933]), 2, 184904],
       ...,
       [13594, 91,
        array([ 54525,  34886,  61307, 130226, 130225,  20136, 148560, 110995,
        38294,  85047,  98436, 155296, 155293,  63525,  11480, 130398,
       149263, 111733, 111722,  40847,  89078,  14280,  27207,  33290,
        33304,   7542,   9145, 113624, 159997, 109851,  71966,   1978,
         9376,   4875, 138910, 115066, 171960,   7371,   6891,  23825,
        99161,  63378,  39967, 163618,  28446,  58308,  93562, 109705,
        61442,   8227, 166182, 163879,  41640,  22422,  96402, 160336,
        30828,  88918, 101293, 101292,  99051, 107968,  49749,  11938,
         5785,  92997,   8508,  33208,  31236,  33278,  26889,  13066,
        51365,  29541,  98993, 116958,  29290, 165525,  17498,  54285,
       155268,  51156, 101336,  11484,  96480,  11936,  32596,  78255,
        25547,  40589, 196437, 192604, 

In [11]:
df = pd.DataFrame(pro_arr)

In [12]:
df.columns = ['userid', 'pathid', 'path', 'length', 'Y']
df.head()

,userid,pathid,path,length,Y
0,35863,10,[144506],2,60286
1,13691,12,[25622],2,39979
2,13731,11,[114933],2,184904
3,13983,49,[103076],2,190034
4,17865,2,[95841],2,150550


In [18]:
df = pd.DataFrame(pro_arr)
df.columns = ['userid', 'pathid', 'path', 'length', 'Y']

print("Sample of training data: ")
print(df.head())

summary = df.groupby('length').count()
summary = pd.DataFrame(summary.drop(columns=['pathid','path']))
summary = summary.reset_index()
summary = summary.drop(['Y'],axis = 1)
summary.columns = ['length', 'count']

slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
slice_table

# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0
print("Here is the slicing table of the data: ")
print(slice_table)

Sample of training data: 
  userid pathid      path length       Y
0  35863     10  [144506]      2   60286
1  13691     12   [25622]      2   39979
2  13731     11  [114933]      2  184904
3  13983     49  [103076]      2  190034
4  17865      2   [95841]      2  150550
Here is the slicing table of the data: 
   length   count  start_index  end_index
0       2  202142            0     202142
1       3   50820       202142     252962
2       4   20257       252962     273219
3       5   10362       273219     283581
4       6    5402       283581     288983
5      10    8525       288983     297508
6      20    3174       297508     300682
7      50     727       300682     301409
8     100     100       301409     301509


In [19]:
def analyze_data(pro_arr):
    '''
    Input:
        1. pro_arr as numpy array
    Output: 
        1. the slicing table of pro_arr
    '''
    df = pd.DataFrame(pro_arr)
    df.columns = ['userid', 'pathid', 'path', 'length', 'Y']

    print("Sample of training data: ")
    print(df.head())

    summary = df.groupby('length').count()
    summary = pd.DataFrame(summary.drop(columns=['pathid','path']))
    summary = summary.reset_index()
    summary = summary.drop(['Y'],axis = 1)
    summary.columns = ['length', 'count']

    slice_table = summary
    slice_table['start_index'] = slice_table['count']
    slice_table['end_index'] = slice_table['count']
    slice_table

    # here is the logic for create the start index + end index
    for i in range(1, len(slice_table)):
        slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

    for i in range(1, len(slice_table)):
        slice_table['start_index'][i] = slice_table['end_index'][i-1]

    slice_table['start_index'][0] = 0
    print("Here is the slicing table of the data: ")
    print(slice_table)
    
    return slice_table
    

In [36]:
# slice_table['end_index'][len(slice_table)-1]
m = pro_arr.shape[0]
np.random.randint(0,m)

167586

In [139]:
def next_batch(batch_size, pro_arr, slice_table):
    '''
    Input:
        1. batch_size as integer
        2. pro_arr as numpy array, with columns: 
            [0] userid as integer
            [1] pathid as integer
            [2] path as numpy array containing itemids as integers
            [3] session_length as integer = length_input + length_output = length_input + 1
            [4] label (output itemid) as integer
    Output:
        1. next_batch_userids as numpy array, with size: [batch_size, ] or [batch_size, num_time_step]
        2. next_batch_itemids as numpy array, with size: [batch_size, num_time_step]
        3. next_batch_y as numpy array, with size: [batch_size, ]
    '''
    # first step: use probability to random select a session length to generate the next batch
    m = pro_arr.shape[0]
    rand = np.random.randint(0,m)
    length = 0
    start_index = 0
    end_index = 0
    for i in range(len(slice_table)):
        if rand >= slice_table['start_index'][i] and rand < slice_table['end_index'][i]:
            length = slice_table['length'][i]
            start_index = slice_table['start_index'][i]
            end_index = slice_table['end_index'][i]
            break
    print("generating next batch with session length t_step =", length)
    next_batchids = np.random.randint(start_index, end_index, batch_size)
    next_batch_userids = pro_arr[next_batchids][:,0]
    next_batch_itemids = np.array(pro_arr[next_batchids][:,2].tolist())
    next_batch_y = pro_arr[next_batchids][:,-1]
    
    return next_batch_userids, next_batch_itemids, next_batch_y    

In [ ]:
# step 1: decide a length for the session
# we are going to take session length <= 18 
